In [ ]:
El script lee un archivo CSV con información de fagos (filtrado previamente), extrae los identificadores GenBank 
desde la columna "Virus GENBANK accession", y descarga automáticamente las secuencias genómicas en formato .fasta 
desde la base de datos de nucleótidos de NCBI utilizando la librería `Biopython` (módulo `Entrez`). Soporta accesiones 
múltiples por fila, evita descargas duplicadas comprobando si el archivo ya existe, y guarda los genomas descargados en 
una carpeta local llamada `genomas_fagos`. Además, incluye una pausa entre descargas para respetar las políticas de uso 
de NCBI y minimizar errores por exceso de peticiones.

In [5]:
!pip install biopython

In [2]:
import sys
!{sys.executable} -m pip install biopython

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 17.6 MB/s eta 0:00:00a 0:00:01


In [1]:
import pandas as pd
import os
from Bio import Entrez
from time import sleep

In [2]:
# Configura tu email (requerido por NCBI)
Entrez.email = "m.garciacervera@um.es"

# Ruta al CSV filtrado
csv_path = "/home/alumno08/TFM/fase 2/fagos_filtrados.csv"
output_dir = "/home/alumno08/TFM/fase 2/genomas_fagos"

# Crear carpeta de salida si no existe
os.makedirs(output_dir, exist_ok=True)

# Leer el CSV
df = pd.read_csv(csv_path)

# Columna con accesos GenBank (pueden ser múltiples separados por ';')
col_accesiones = "Virus GENBANK accession"

# Contador de descargas exitosas
descargados = 0

# Iterar por cada fila
for idx, fila in df.iterrows():
    accesiones = str(fila.get(col_accesiones, "")).strip()
    if accesiones == "" or accesiones.lower() == "nan":
        continue

    # Separar accesiones (a veces hay múltiples, tipo: "NC_001416; NC_001417")
    for acc in accesiones.replace(";", ",").split(","):
        acc = acc.strip()
        if acc == "":
            continue

        output_file = os.path.join(output_dir, f"{acc}.fasta")

        # Evitar repetir descargas
        if os.path.exists(output_file):
            continue

        try:
            # Descargar desde NCBI
            with Entrez.efetch(db="nucleotide", id=acc, rettype="fasta", retmode="text") as handle:
                fasta_data = handle.read()

            # Guardar el archivo
            with open(output_file, "w") as f:
                f.write(fasta_data)

            print(f"✅ Descargado: {acc}")
            descargados += 1

            # Esperar un poco para no sobrecargar NCBI (¡muy importante!)
            sleep(0.4)

        except Exception as e:
            print(f"⚠️ Error con {acc}: {e}")

print(f"\n✅ Descargas completas: {descargados}")

✅ Descargado: PP079415
✅ Descargado: MW960043
✅ Descargado: MT951568
✅ Descargado: KF626666
✅ Descargado: MZ326858
✅ Descargado: MK903280
✅ Descargado: MZ326854
✅ Descargado: PP079416
✅ Descargado: OK018136
✅ Descargado: OP583591
✅ Descargado: KU595432
✅ Descargado: KU595433
✅ Descargado: ON911539
✅ Descargado: ON932078
✅ Descargado: ON932083
✅ Descargado: OK258138
✅ Descargado: OP548052
✅ Descargado: OP583594
✅ Descargado: MK903278
✅ Descargado: KF626667
✅ Descargado: ON960072
✅ Descargado: PP079417
✅ Descargado: MN478375
✅ Descargado: KU197013
✅ Descargado: MF375456
✅ Descargado: MW831865
✅ Descargado: OM735686
✅ Descargado: OR209256
✅ Descargado: OK490494
✅ Descargado: PP079413
✅ Descargado: KT184661
✅ Descargado: PP226938
✅ Descargado: KT949345
✅ Descargado: ON365567
✅ Descargado: MF683623
✅ Descargado: MK838107
✅ Descargado: OP797796
✅ Descargado: OR261031
✅ Descargado: OM654404
✅ Descargado: MT682386
✅ Descargado: MT682390
✅ Descargado: LN878149
✅ Descargado: MW882933
✅ Descargad

In [2]:
#Para quitar los genomas parciales

# Ruta a tu CSV y carpeta con FASTA
csv_path = "/home/alumno08/TFM/fase 2/fagos_filtrados.csv"
fasta_dir = "/home/alumno08/TFM/fase 2/genomas_fagos"

# Leer el CSV
df = pd.read_csv(csv_path)

# Filtrar los que tienen genoma parcial
df_parciales = df[df["Genome coverage"] == "Partial genome"]

# Extraer accesiones GenBank (puede haber múltiples por fila)
accesiones = set()
for accs in df_parciales["Virus GENBANK accession"].dropna():
    for acc in str(accs).replace(";", ",").split(","):
        accesiones.add(acc.strip())

# Mostrar accesiones encontradas
print(f"🧬 Genomas parciales detectados: {len(accesiones)}")

# Eliminar los archivos .fasta asociados
eliminados = 0
for acc in accesiones:
    fasta_path = os.path.join(fasta_dir, f"{acc}.fasta")
    if os.path.exists(fasta_path):
        os.remove(fasta_path)
        print(f"🗑️ Eliminado: {fasta_path}")
        eliminados += 1

print(f"\n✅ Total de archivos .fasta eliminados: {eliminados}")

🧬 Genomas parciales detectados: 54
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/MN689513.fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/CP000031 (2001541.2913924).fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/GU943054.fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/GU943065.fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/MG387042.fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/KJ127304.fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/MF498775.fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/JN672684.fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/MF001354.fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/GU943073.fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/RNA2: AF246939.fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/MG676224.fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/genomas_fagos/MG592603.fasta
🗑️ Eliminado: /home/alumno08/TFM/fase 2/geno

In [ ]:
#Número total de genomas... 6730

(base) [alumno08@eck fase 2]$ ls /home/alumno08/TFM/fase\ 2/genomas_fagos/*.fasta | wc -l
6730